In [15]:
from pandas import read_csv
import math
import numpy as np
import pandas as pd
initial= read_csv("b.tsv",header=None,sep="\t")
k=3 # number of folds
n=initial.shape[0] # number of total rows
l= math.ceil(n/k) # maximum number of rows in each fold
f=initial.shape[1] # number of features
maxNei=3 # maximum number of neighbours that we try, nei is the number of neighbours for KNN that changes from 1 to maxNei
folds= np.empty((k,l,f)) # creating empty folds, we have k fold, each has l rows and f column
folds[:]=np.nan
# if need to shuffle use this:
# suffle them with sklearn 
# from sklearn import utils
# initial=utils.shuffle(initial)
classA= initial.loc[initial[f-1]==0] # the one with 0 in the last column are class A
classB= initial.loc[initial[f-1]==1]
ca=classA.values # convert pandas Data frame to numpy array
cb=classB.values
sizeA=ca.shape[0]
sizeB=cb.shape[0]
for i in range(sizeA): # asigning almost equal number of class A and B to folds
        folds[i%k,i//k,:]=ca[i,:]
sizeT=sizeA+sizeB
for i in range(sizeA,sizeT,1): # we start from sizeA to continue with the next fold, not necceserily the first one
        folds[i%k,i//k,:]=cb[i-sizeA,:]
# Untill here we read the data, devided into k fold, each fold has same number of class A and B
# print(folds) # check the results
#################################################################
allScores=pd.DataFrame(np.empty((f-1,maxNei)))
import pandas as pd
for kk in range(k):
    #### the following repeats for all k folds
    # Convert the selected fold , kk, to test and train sets
    print("******************************Starting calculation for this fold*******************************")
    dfTest=pd.DataFrame(folds[kk,:,:]).dropna().reset_index(drop=True)
    dfTrain= pd.DataFrame(np.delete(folds, kk, axis=0).ravel().reshape((k-1)*l,f)).dropna().reset_index(drop=True)
#     print("{} rd test set is".format(kk)) # just for check
#     print(dfTest)    
    ############  Now for this fold we have test and training data so
    ############  we start by selecting first feature ( the most important one) in the next loop
    ############  and for that we perform KNN several times
    bestScore=0 # setting the best KNN score for start
    bestP=0 # shows a p that has the highest score
    for p in range(0,f-1): # the loop for feature selection
        print("=== for {} features:===".format(p+1))
        # Defining test and training data
        X_train=(dfTrain.loc[:,0:p]) 
        X_test=(dfTest.loc[:,0:p])
        y_train=dfTrain[f-1]
        y_test=dfTest[f-1]
    #     print(X_train) # print to check
    #     print(X_test)
    #     print(y_train)
    #     print(y_test)
        from sklearn.neighbors import KNeighborsClassifier
        for nei in range(1,maxNei):
            print("---- Performing KNN, {} is the number of neighbours:----".format(nei))
            neigh = KNeighborsClassifier(n_neighbors=nei)
            neigh.fit(X_train, y_train)
            score=neigh.score(X_test, y_test) # here y_test is equal to y_actual
            # storing the score in allScore array:
            allScores.loc[p,nei]=score+allScores.loc[p,nei]
            print("Best score untill now:",bestScore)
            print("score difference {(newscore-bestScore)/newscore)} is",((score-bestScore)/score)," And current number of neighbours is", nei)
            if (((score-bestScore)/score)>0.1): # a threshold that if the score improves significantly we update old p and nei
                bestK=nei # we have a new bestK, which is number of neghbours
                bestScore=score
                bestP=p
                print("New bestP:",bestP, "    and best K is:---->",nei," New best score:",bestScore)
            print("-------End of this KNN loop-----")
        print("===============")
        print("")
        print("Best p till now",bestP)
        print("")
    print("For this fold : best P:",bestP, "    and best K is:",nei,"  best score:",bestScore)
    print("******************************End of the fold calculations*******************************")
    print("")
## Now we have found number of neighbours and p for each fold. According to the algorithm we need to consider the average
## Change the order of the loop


# This matrix contains some of all accuracies for all p and knn neighbours. The indecies of maximum are our answer 
maxi=allScores.values.max()
print(maxi)
# print(allScores.loc[allScores.loc[:,:] == maxi])
# print(" Optimum number of KNN number , is: ",allScores[maxi].index(maxi))
# print(" Optimum number of features, p, is: ",allScores.idmax(axis=0))
print(allScores)

******************************Starting calculation for this fold*******************************
=== for 1 features:===
---- Performing KNN, 1 is the number of neighbours:----
Best score untill now: 0
score difference {(newscore-bestScore)/newscore)} is 1.0  And current number of neighbours is 1
New bestP: 0     and best K is:----> 1  New best score: 0.25
-------End of this KNN loop-----
---- Performing KNN, 2 is the number of neighbours:----
Best score untill now: 0.25
score difference {(newscore-bestScore)/newscore)} is 0.5  And current number of neighbours is 2
New bestP: 0     and best K is:----> 2  New best score: 0.5
-------End of this KNN loop-----

Best p till now 0

=== for 2 features:===
---- Performing KNN, 1 is the number of neighbours:----
Best score untill now: 0.5
score difference {(newscore-bestScore)/newscore)} is 0.0  And current number of neighbours is 1
-------End of this KNN loop-----
---- Performing KNN, 2 is the number of neighbours:----
Best score untill now: 0.5